# Hacking Dask clusters

In this notebook we'll cover:

- Cluster overview
- Inspecting a cluster's state
- Dynamic hooks: worker and scheduler plugins
- Handlers
- Coordination primatives: `Lock`, `Event`, ...

# Cluster overview

In this section we'll discuss:

1. The different components which make up a Dask cluster
2. How to launch a cluster

## Components of a cluster

A Dask cluster is composed of three different types of objects:

1. **Scheduler**: A single, centralized scheduler which responds to requests for computations and manages ...
2. **Workers**: One or more worker processes which compute, store, and serve computational results
3. **Clients**: One or more client objects which are the user-facing entry point to interact with the cluster

<img src="images/dask-cluster.svg"
     width="60%"
     alt="Dask components\">

## Deploying Dask clusters

Deploying a Dask cluster means launching scheduler, worker, and client processes and setting up the appropriate network connections so these processes can communicate with one another. Dask clusters can be lauched in a few different ways which we highlight in the following sections.

### Manual setup

Launch a scheduler process using the `dask-scheduler` command line utility:

```terminal
$ dask-scheduler
Scheduler at:   tcp://192.0.0.100:8786
```

and then launch several workers by using the `dask-worker` command and providing them the address of the scheduler they should connect to:

```terminal
$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.1:12345
Registered to:    tcp://192.0.0.100:8786

$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.2:40483
Registered to:    tcp://192.0.0.100:8786

$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.3:27372
Registered to:    tcp://192.0.0.100:8786
```

### Python API (advanced)

⚠️ **Warning**: Creating `Scheduler` / `Worker` objects explicitly in Python is only needed in rare circumstances and is intended for expert users ⚠️

In [ ]:
from dask.distributed import Scheduler, Worker, Client

# Launch a scheduler
async with Scheduler() as scheduler: # Launch a scheduler
    # Launch a worker which connects to the scheduler
    async with Worker(scheduler.address) as worker:
        # Launch a client which connects to the scheduler
        async with Client(scheduler.address, asynchronous=True) as client:
            result = await client.submit(sum, range(100))
            print(f"{result = }")

### Cluster managers (recommended)

In [ ]:
from dask.distributed import LocalCluster

# Launch a scheduler and 4 workers on my local machine
cluster = LocalCluster(n_workers=4, threads_per_worker=2)

In [ ]:
# scale up to 10 workers
cluster.scale(10)

In [ ]:
# scale down to 2 workers
cluster.scale(2)

In [ ]:
cluster.close()

There are several projects in the Dask ecosystem for easily deploying clusters on commonly used computing resources:

- [Dask-Kubernetes](https://kubernetes.dask.org/en/latest/) for deploying Dask using native Kubernetes APIs
- [Dask-Cloudprovider](https://cloudprovider.dask.org/en/latest/) for deploying Dask clusters on various cloud platforms (e.g. AWS, GCP, Azure, etc.)
- [Dask-Yarn](https://yarn.dask.org/en/latest/) for deploying Dask on YARN clusters
- [Dask-MPI](http://mpi.dask.org/en/latest/) for deploying Dask on existing MPI environments
- [Dask-Jobqueue](https://jobqueue.dask.org/en/latest/) for deploying Dask on job queuing systems (e.g. PBS, Slurm, etc.)

Launching clusters with any of these projects follows a similar pattern as using Dask's built-in `LocalCluster`:

```python
# Launch a Dask cluster on a Kubernetes cluster
from dask_kubernetes import KubeCluster
cluster = KubeCluster(...)

# Launch a Dask cluster on AWS Fargate
from dask_cloudprovider.aws import FargateCluster
cluster = FargateCluster(...)

# Launch a Dask cluster on a PBS job queueing system
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
```

We'll discuss this more throughout the course of this tutorial, but if you're interested in learning more about the various steps involved in computing a task we recommended checking out the [*Journey of a Task*](https://distributed.dask.org/en/latest/journey.html) page in the Dask documentation.

# Inspecting a cluster's state

In this section we'll:

1. Familiarize ourselves with Dask's scheduler and worker processes
2. Explore the various state that's tracked throughout the cluster

Let's start by creating a local cluster and perform a small computation.

In [ ]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
import dask.array as da

x = da.random.random((100, 100), chunks=(50, 50))
x = x.persist()

One of the nice things about a `LocalCluster` is it gives us direct access the `Scheduler` Python object. This allows us to easily inspect the scheduler directly.

In [ ]:
scheduler = cluster.scheduler
type(scheduler)

ℹ️ Note that often times you won't have direct access to the `Scheduler` Python object (e.g. when the scheduler is running on separate machine). In these cases it's still possible to inspect the scheduler and we will discuss how to do this later on.

The scheduler tracks **a lot** of state. Let's start to explore the scheduler to get a sense for what information it keeps track of.

In [ ]:
scheduler.address   # Scheduler's address

In [ ]:
scheduler.time_started   # Time the scheduler was started

In [ ]:
dict(scheduler.workers)

In [ ]:
worker_state = next(iter(scheduler.workers.values()))
worker_state

In [ ]:
type(worker_state)

Let's take a look at the `WorkerState` attributes

In [ ]:
[attr for attr in dir(worker_state) if not attr.startswith("_")]

In [ ]:
worker_state.address

In [ ]:
worker_state.status

In [ ]:
worker_state.processing

Workers periodically send a message 

In [ ]:
worker_state.last_seen

In [ ]:
import time

for _ in range(10):
    print(f"{worker_state.last_seen = }")
    time.sleep(0.5)

In [ ]:
worker_state.metrics

In [ ]:
scheduler.total_nthreads

# Scheduler and worker plugins

# Coordination Primitives